In [21]:
import UninformedSearchWithDB as UDB
import importlib
import mysql.connector
import json
import urllib.request
import csv
importlib.reload(UDB)

<module 'UninformedSearchWithDB' from '/Users/sreejithmenon/Google Drive/CS material/DBMS CS 581/ParkingSimulation/script/UninformedSearchWithDB.py'>

In [5]:
# cnxObj = UDB.connectToMySQL()
# UDB.closeConnection(cnxObj)
edgeData = UDB.getParkingDataUninform() # all parking blocks


In [35]:
# driving distance calculations - from intersections to parking blocks
blockToBlock = []
for i in range(37,len(edgeData)):
    ixcord = float(edgeData[i]['midptLat'])
    iycord = float(edgeData[i]['midptLong'])
    point1 = UDB.Point(ixcord,iycord)
    for j in range(len(edgeData)):
        jxcord = float(edgeData[j]['midptLat'])
        jycord = float(edgeData[j]['midptLong'])
        point2 = UDB.Point(jxcord,jycord)
        distance,time = point1.distanceFromAPI(point2)
        if distance == None:
            print("exiting at i=%d j=%d" %(i,j))
            break
        else:
            blockDict = {'block1':edgeData[i]['blockID'],'sourceLat':edgeData[i]['midptLat'],'sourceLong':edgeData[i]['midptLong'],'block2':edgeData[j]['blockID'],'destLat':edgeData[j]['midptLat'],'destLong':edgeData[j]['midptLong'],'distance':distance,'time':time}
            blockToBlock.append(blockDict)
            break
    break

In [ ]:
writeFL = open("../data/blockToBlock.csv","w")
head = ['start_block','end_block','distance','time']
writer = csv.writer(writeFL)
writer.writerow(head)
for row in blockToBlock:
    writer.writerow([row['block1'],row['sourceLat'],row['sourceLong'],row['block2'],row['destLat'],row['destLong'],row['distance'],row['time']])
writeFL.close()

In [6]:
# Get all parking block information
cnx = UDB.connectToMySQL()
cursor = cnx.cursor()
query = ("SELECT node_id,latitude,longitude FROM parkingproject.nodes;")
cursor.execute(query)

nodeData = []
for node_id,latitude,longitude in cursor:
    nodeDataDict = {}
    nodeDataDict['node_id'] = node_id
    nodeDataDict['latitude'] = latitude
    nodeDataDict['longitude'] = longitude
    nodeData.append(nodeDataDict)

UDB.closeConnection(cnx)

'Success. Connection closed.'

In [45]:
# logic to calculate driving distances from every parking block to each other
edgeToBlock = []
for i in range(37,len(nodeData)):
    ixcord = float(nodeData[i]['latitude'])
    iycord = float(nodeData[i]['longitude'])
    point1 = UDB.Point(ixcord,iycord)
    for j in range(len(edgeData)):
        jxcord = float(edgeData[j]['midptLat'])
        jycord = float(edgeData[j]['midptLong'])
        point2 = UDB.Point(jxcord,jycord)
        distance,time = point1.distanceFromAPI(point2)
        if distance == None:
            print("exiting at i=%d j=%d" %(i,j))
            break
        else:
            blockDict = {'block1':nodeData[i]['node_id'],
                         'sourceLat':nodeData[i]['latitude'],
                         'sourceLong':nodeData[i]['longitude'],
                         'block2':edgeData[j]['blockID'],
                         'destLat':edgeData[j]['midptLat'],
                         'destLong':edgeData[j]['midptLong'],
                         'distance':distance,
                         'time':time}
            edgeToBlock.append(blockDict)

In [47]:
writeFL = open("../data/edgeToBlock1.csv","w")
head = ['node_id','nodeLat','nodeLong','block_id','edgeLat','edgeLong','distance','time']
writer = csv.writer(writeFL)
writer.writerow(head)
for row in edgeToBlock:
    writer.writerow([row['block1'],row['sourceLat'],row['sourceLong'],row['block2'],row['destLat'],row['destLong'],row['distance'],row['time']])
writeFL.close()

In [22]:
# Walking distance calculations from parking blocks to intersections

edgeToBlockWalk = []
for i in range(60,len(edgeData)):
    ixcord = float(edgeData[i]['midptLat'])
    iycord = float(edgeData[i]['midptLong'])
    point1 = UDB.Point(ixcord,iycord)
    for j in range(len(nodeData)):
        jxcord = float(nodeData[j]['latitude'])
        jycord = float(nodeData[j]['longitude'])
        point2 = UDB.Point(jxcord,jycord)
        distance,time = point1.distanceFromAPI(point2)
        if distance == None:
            print("exiting at i=%d j=%d" %(i,j))
            break
        else:
            blockDict = {'block2':nodeData[j]['node_id'],
                         'block1':edgeData[i]['blockID'],
                         'distance':distance,
                         'time':time}
            edgeToBlockWalk.append(blockDict)

In [23]:
edgeToBlockWalk[0]

{'block1': 847021, 'block2': 7001, 'distance': 717, 'time': 530}

In [24]:
writeFL = open("../data/blockToEdgeWalk1.csv","w")
head = ['block_id','node_id','distance','time']
writer = csv.writer(writeFL)
writer.writerow(head)
for row in edgeToBlockWalk:
    writer.writerow([row['block1'],row['block2'],row['distance'],row['time']])
writeFL.close()